In [1]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer , WordNetLemmatizer
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
import pickle
import warnings
warnings.filterwarnings('ignore')

In [10]:
df = pd.read_csv(r"E:\EMAIL JUPYTER NOTEBOOK\spam.csv" ,encoding='latin-1')

In [11]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [12]:
df.shape

(5572, 5)

In [13]:
#1. Data Cleaning
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
v1            5572 non-null object
v2            5572 non-null object
Unnamed: 2    50 non-null object
Unnamed: 3    12 non-null object
Unnamed: 4    6 non-null object
dtypes: object(5)
memory usage: 217.7+ KB


In [14]:
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [15]:
df.sample(5)

,v1,v2
3443,ham,Yes but I don't care cause I know its there!
3865,ham,"Theoretically yeah, he could be able to come"
5343,ham,No go. No openings for that room 'til after th...
2206,spam,"URGENT! Your mobile No 07xxxxxxxxx won a å£2,0..."
3595,ham,Good morning princess! Happy New Year!


In [16]:
# renaming the cols
df.rename(columns={'v1':'label','v2':'message'},inplace=True)
df.sample(5)

,label,message
4157,ham,It certainly puts things into perspective when...
3513,ham,I always chat with you. In fact i need money c...
4326,ham,Dude got a haircut. Now its breezy up there
3215,spam,URGENT! We are trying to contact U. Todays dra...
2490,ham,Dun b sad.. It's over.. Dun thk abt it already...


In [17]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [18]:
df['label'] = encoder.fit_transform(df['label'])

In [19]:
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [20]:
df.shape

(5572, 2)

In [21]:
lamitizer = WordNetLemmatizer()

In [22]:
train = []
for i in range(len(df)):
    review = re.sub('^[a-zA-Z]' , ' ' , df['message'][i])
    review = review.lower()
    review = review.split()
    review = [lamitizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    train.append(review)

In [23]:
print(train[0])

jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...


In [24]:
cv = CountVectorizer(max_features=5000)
x = cv.fit_transform(train).toarray()

In [25]:
y = pd.get_dummies(df['label'])
y.head()

,0,1
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [26]:
y = y.iloc[:,1].values
print(y)

[0 0 1 ... 0 0 0]


In [27]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size = 0.20,random_state = 0)

In [28]:
from sklearn.naive_bayes import MultinomialNB

In [29]:
mb = MultinomialNB()
mb.fit(train_x,train_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [30]:
pred = mb.predict(test_x)
print(pred)

[0 0 0 ... 0 0 0]


In [31]:
cm = confusion_matrix(pred,test_y)
print(cm)

[[944   8]
 [  5 158]]


In [32]:
acc = accuracy_score(pred,test_y)
acc

0.9883408071748879

In [33]:
pickle.dump(cv,open('transform.pkl','wb'))

In [34]:
pickle.dump(mb,open('nlp model.pkl','wb'))